In [1]:

import os
import pandas as pd


In [6]:
import os
import pandas as pd

class ExcelReader:
    def __init__(self, base_path: str, year: int = 2018, aggregation: str = 'S68'):
        self.base_path = base_path
        self.year = year
        self.aggregation = aggregation
        self.file_path = self._discover_excel_file()
        self.load_sheets()

    def _discover_excel_file(self):
        # Descobrir o arquivo Excel com base na agregação e no ano
        target_path = os.path.join(self.base_path, 'data', 'input_output_matrix', self.aggregation, str(self.year))
        print(f"Looking for directory: {target_path}")

        if not os.path.isdir(target_path):
            raise FileNotFoundError(f"The directory '{target_path}' does not exist.")
        
        excel_files = [os.path.join(target_path, file) for file in os.listdir(target_path) if file.endswith('.xlsx')]
        
        if len(excel_files) != 1:
            raise FileNotFoundError(f"Expected exactly one Excel file in '{target_path}', found {len(excel_files)}.")
        
        return excel_files[0]

    def load_sheets(self):
        if self.file_path:
            excel_data = pd.ExcelFile(self.file_path)
            
            # Verificar se a planilha 'data' existe
            if 'data' in excel_data.sheet_names:
                df = pd.read_excel(
                    self.file_path,
                    sheet_name='data',
                    header=None,
                    skiprows=4,  # Pular as linhas antes do início dos dados
                    usecols='D:',  # Usar colunas a partir de 'D'
                    index_col=None
                )
                
                if df.shape[1] > 2:
                    df.index = pd.MultiIndex.from_frame(df.iloc[:, 0:2], names=['Index1', 'Index2'])
                    df = df.iloc[:, 2:]
                    
                    # Nomear o atributo como 'iom_{aggregation}_{year}_data'
                    formatted_sheet_name = f"iom_{self.aggregation}_{self.year}_data"
                    setattr(self, formatted_sheet_name, df)
                else:
                    print(f"Warning: Sheet 'data' in file '{self.file_path}' does not have sufficient columns.")
            else:
                raise ValueError(f"Sheet 'data' not found in file '{self.file_path}'")

    def get_sheet_data(self):
        formatted_sheet_name = f"iom_{self.aggregation}_{self.year}_data"
        return getattr(self, formatted_sheet_name, None)

    def get_all_sheet_names(self):
        return [attr for attr in dir(self) if isinstance(getattr(self, attr), pd.DataFrame)]

# Ajustar o caminho base considerando o local do Jupyter Notebook
notebook_directory = os.path.dirname(os.path.abspath('read_matrix.ipynb'))
base_path = os.path.join(notebook_directory, 'data', 'input_output_matrix')
display(base_path)
# Criar uma instância de ExcelReader
year = 2018  # Atualize o ano conforme necessário
aggregation = 'S68'  # Atualize a agregação conforme necessário
excel_reader = ExcelReader(base_path, year=year, aggregation=aggregation)

# Recuperar e imprimir os nomes de todas as planilhas
sheet_names = excel_reader.get_all_sheet_names()
print(f"Sheet names: {sheet_names}")

# Exemplo: Recuperar dados da planilha específica
data = excel_reader.get_sheet_data()
if data is not None:
    print(f"Data from sheet 'data':")
    print(data.head())
else:
    print(f"No data found for sheet 'data'")




'c:\\Users\\guitz\\OneDrive\\Área de Trabalho\\input_output\\input_output\\data\\input_output_matrix'

Looking for directory: c:\Users\guitz\OneDrive\Área de Trabalho\input_output\input_output\data\input_output_matrix\data\input_output_matrix\S68\2018


FileNotFoundError: The directory 'c:\Users\guitz\OneDrive\Área de Trabalho\input_output\input_output\data\input_output_matrix\data\input_output_matrix\S68\2018' does not exist.

In [5]:
# Create an instance of ExcelReader
year = 2011
aggregation = 'S68'
excel_reader = ExcelReader(base_path, year=year, aggregation=aggregation)

Looking for directory: ..\data\input_output_matrix\data\input_output_matrix\S68\2011


FileNotFoundError: The directory '..\data\input_output_matrix\data\input_output_matrix\S68\2011' does not exist.

In [ ]:
file_path = '..//data//'S68'// 2018//MIP-BR-CN10-68S-2014.xlsx'
df = pd.read_excel(file_path)
# excel_reader = ExcelReader(file_path)

In [8]:
file_path = '..//data//S68//2018//MIP-BR-CN10-68S-2014.xlsx'


In [9]:
df = pd.read_excel(file_path)


FileNotFoundError: [Errno 2] No such file or directory: '..//data//S68//2018//MIP-BR-CN10-68S-2014.xlsx'